In [631]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import matplotlib.pyplot as plt

In [632]:
data = pd.read_csv("insurance.csv")

In [633]:
sex_map = {"female" : 0, "male" : 1}
smoker_map = {"yes" : 0, "no" : 1}
data['sex'] = data.sex.map(sex_map)
data['smoker'] = data.smoker.map(smoker_map)

In [634]:
data_type = data.dtypes.reset_index()
data_type.columns = ["column_name","column_type"]
data_type

,column_name,column_type
0,age,int64
1,sex,int64
2,bmi,float64
3,children,int64
4,smoker,int64
5,region,object
6,expenses,float64


In [635]:
data_type[data_type.column_type == "object"].column_name.tolist()
data = pd.get_dummies(data, columns = data_type[data_type.column_type == "object"].column_name.tolist())

In [636]:
data_x = data.drop(["expenses"],axis = 1)
data_y = data["expenses"]
data_x,data_y

(      age  sex      bmi  children  smoker  region_northeast  region_northwest  \
 0      19    0 27.90000         0       0                 0                 0   
 1      18    1 33.80000         1       1                 0                 0   
 2      28    1 33.00000         3       1                 0                 0   
 3      33    1 22.70000         0       1                 0                 1   
 4      32    1 28.90000         0       1                 0                 1   
 ...   ...  ...      ...       ...     ...               ...               ...   
 1333   50    1 31.00000         3       1                 0                 1   
 1334   18    0 31.90000         0       1                 1                 0   
 1335   18    0 36.90000         0       1                 0                 0   
 1336   21    0 25.80000         0       1                 0                 0   
 1337   61    0 29.10000         0       0                 0                 1   
 
       region_

In [637]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data_x,data_y,train_size = .7,shuffle=True, random_state=1)

#train_x['train_test_label'] = 1
#test_x['train_test_label'] = 0

#data_x_1 = pd.concat([train_x, test_x], ignore_index=True)

In [638]:
#train_x = data_x_1[data_x_1.train_test_label==1]
#test_x = data_x_1[data_x_1.train_test_label==0]
#train_x.drop(["train_test_label"],axis = 1, inplace = True)
#test_x.drop(["train_test_label"],axis = 1, inplace = True)
train_x.shape,test_x.shape, train_y.shape, test_y.shape

((936, 9), (402, 9), (936,), (402,))

In [639]:
train_x

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
744,50,1,26.40000,0,1,0,1,0,0
363,21,0,26.40000,1,1,0,0,0,1
10,25,1,26.20000,0,1,1,0,0,0
970,50,0,28.20000,3,1,0,0,1,0
634,51,1,39.70000,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
715,60,1,28.90000,0,1,0,0,0,1
905,26,0,29.40000,2,1,1,0,0,0
1096,51,0,35.00000,2,0,1,0,0,0
235,40,0,22.20000,2,0,0,0,1,0


In [640]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(train_x, train_y)
prediction = lr_model.predict(test_x)

In [641]:
compare_dataframe = pd.DataFrame({"prediction": prediction, "actual" : test_y})

from sklearn.metrics import mean_squared_error, r2_score

mse_error = mean_squared_error(compare_dataframe.actual,compare_dataframe.prediction)
mse_error,compare_dataframe, print(" R^2 Score: {:.5f}".format(lr_model.score(test_x, test_y)))

 R^2 Score: 0.74060


(36761225.78409526,
       prediction      actual
 559   4601.37921  1646.43000
 1087 12874.27504 11353.23000
 1020 12574.91140  8798.59000
 460  13187.34383 10381.48000
 802    628.64799  2103.08000
 ...          ...         ...
 323  16209.90521 11566.30000
 1268  3862.63052  1880.49000
 134   3278.27418  2457.21000
 1274 27076.10826 17043.34000
 876   9505.95402 26140.36000
 
 [402 rows x 2 columns],
 None)

In [642]:
train_x_copy = train_x.copy()
#col_names = data_type[data_type.column_type != "object"].column_name.tolist()[:-1]
#features = data_scaled[col_names]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_x_copy.values)

train_x = pd.DataFrame(scaler.transform(train_x), columns=train_x.columns)
test_x = pd.DataFrame(scaler.transform(test_x), columns=test_x.columns)

In [643]:
train_x

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,0.79715,0.97677,-0.70421,-0.90400,0.50966,-0.56254,1.78810,-0.60039,-0.58722
1,-1.27109,-1.02379,-0.70421,-0.08568,0.50966,-0.56254,-0.55925,-0.60039,1.70294
2,-0.98581,0.97677,-0.73708,-0.90400,0.50966,1.77764,-0.55925,-0.60039,-0.58722
3,0.79715,-1.02379,-0.40838,1.55097,0.50966,-0.56254,-0.55925,1.66559,-0.58722
4,0.86847,0.97677,1.48163,-0.08568,0.50966,-0.56254,-0.55925,-0.60039,1.70294
...,...,...,...,...,...,...,...,...,...
931,1.51034,0.97677,-0.29333,-0.90400,0.50966,-0.56254,-0.55925,-0.60039,1.70294
932,-0.91449,-1.02379,-0.21116,0.73264,0.50966,1.77764,-0.55925,-0.60039,-0.58722
933,0.86847,-1.02379,0.70919,0.73264,-1.96208,1.77764,-0.55925,-0.60039,-0.58722
934,0.08397,-1.02379,-1.39447,0.73264,-1.96208,-0.56254,-0.55925,1.66559,-0.58722


In [644]:
#train_x = data_scaled[data_scaled.train_test_label==1]
#test_x = data_scaled[data_scaled.train_test_label==0]
#train_x.drop(["train_test_label"],axis = 1, inplace = True)
#test_x.drop(["train_test_label"],axis = 1, inplace = True)
train_x.shape,test_x.shape, train_y.shape, test_y.shape

((936, 9), (402, 9), (936,), (402,))

In [645]:
test_x

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,-1.41372,0.97677,0.79137,-0.90400,0.50966,-0.56254,1.78810,-0.60039,-0.58722
1,1.29638,0.97677,0.13397,-0.90400,0.50966,-0.56254,1.78810,-0.60039,-0.58722
2,0.86847,0.97677,1.03789,-0.90400,0.50966,-0.56254,-0.55925,-0.60039,1.70294
3,0.72583,-1.02379,0.97215,1.55097,0.50966,-0.56254,-0.55925,1.66559,-0.58722
4,-1.27109,0.97677,-1.37804,-0.08568,0.50966,-0.56254,-0.55925,-0.60039,1.70294
...,...,...,...,...,...,...,...,...,...
397,1.29638,0.97677,1.67885,-0.90400,0.50966,1.77764,-0.55925,-0.60039,-0.58722
398,-1.34240,-1.02379,0.42980,-0.90400,0.50966,-0.56254,-0.55925,-0.60039,1.70294
399,-1.34240,-1.02379,-0.30977,-0.90400,0.50966,1.77764,-0.55925,-0.60039,-0.58722
400,-0.91449,0.97677,-0.58916,-0.90400,-1.96208,-0.56254,-0.55925,1.66559,-0.58722


In [646]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(train_x, train_y)
prediction = lr_model.predict(test_x)

In [647]:
compare_dataframe = pd.DataFrame({"prediction": prediction, "actual" : test_y})

from sklearn.metrics import mean_squared_error, r2_score

mse_error = mean_squared_error(compare_dataframe.actual,compare_dataframe.prediction)
mse_error,compare_dataframe, print(" R^2 Score: {:.5f}".format(lr_model.score(test_x, test_y)))


 R^2 Score: 0.74013


(36827328.26176622,
       prediction      actual
 559   4843.67486  1646.43000
 1087 13099.67486 11353.23000
 1020 12939.67486  8798.59000
 460  13067.67486 10381.48000
 802    843.67486  2103.08000
 ...          ...         ...
 323  16235.67486 11566.30000
 1268  3787.67486  1880.49000
 134   2795.67486  2457.21000
 1274 27531.67486 17043.34000
 876   9355.67486 26140.36000
 
 [402 rows x 2 columns],
 None)

In [648]:
from sklearn.linear_model import ElasticNet
my_alpha = 0.1
my_model = ElasticNet(alpha = my_alpha)
my_model.fit(train_x, train_y)
prediction = my_model.predict(test_x)

In [649]:
compare_dataframe = pd.DataFrame({"prediction": prediction, "actual" : test_y})

from sklearn.metrics import mean_squared_error, r2_score

mse_error = mean_squared_error(compare_dataframe.actual,compare_dataframe.prediction)
mse_error,compare_dataframe, print(" R^2 Score: {:.5f}".format(lr_model.score(test_x, test_y)))

 R^2 Score: 0.74013


(36909350.12835885,
       prediction      actual
 559   5011.14155  1646.43000
 1087 12901.14557 11353.23000
 1020 12643.57473  8798.59000
 460  13229.44802 10381.48000
 802   1232.35089  2103.08000
 ...          ...         ...
 323  16116.21973 11566.30000
 1268  4260.83066  1880.49000
 134   3700.48461  2457.21000
 1274 26486.71339 17043.34000
 876   9639.63533 26140.36000
 
 [402 rows x 2 columns],
 None)

NameError: name 'model' is not defined

In [600]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Binary encoding
    df['sex'] = df['sex'].replace({'female': 0, 'male': 1})
    df['smoker'] = df['smoker'].replace({'no': 0, 'yes': 1})
    
    # One-hot encoding
    region_dummies = pd.get_dummies(df['region'], prefix='region')
    df = pd.concat([df, region_dummies], axis=1)
    df = df.drop('region', axis=1)
    
    # Split df into X and y
    y = df['expenses']
    X = df.drop('expenses', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    
    return X_train, X_test, y_train, y_test

In [601]:
train_x, test_x, train_y, test_y = preprocess_inputs(data)

In [602]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

models = {
    "                     Linear Regression": LinearRegression(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                     Gradient Boosting": GradientBoostingRegressor()
}

for name, model in models.items():
    model.fit(train_x, train_y)
    print(name + " trained.")

                     Linear Regression trained.
                   K-Nearest Neighbors trained.


C:\Users\shuba\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.


In [603]:
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(test_x, test_y)))

                     Linear Regression R^2 Score: 0.74013
                   K-Nearest Neighbors R^2 Score: 0.78952
                        Neural Network R^2 Score: -1.14292
Support Vector Machine (Linear Kernel) R^2 Score: -1.07099
   Support Vector Machine (RBF Kernel) R^2 Score: -0.11296
                         Decision Tree R^2 Score: 0.71896
                         Random Forest R^2 Score: 0.83147
                     Gradient Boosting R^2 Score: 0.86200


In [604]:
train_x

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,0.79715,0.97677,-0.70421,-0.90400,-0.50966,-0.56254,1.78810,-0.60039,-0.58722
1,-1.27109,-1.02379,-0.70421,-0.08568,-0.50966,-0.56254,-0.55925,-0.60039,1.70294
2,-0.98581,0.97677,-0.73708,-0.90400,-0.50966,1.77764,-0.55925,-0.60039,-0.58722
3,0.79715,-1.02379,-0.40838,1.55097,-0.50966,-0.56254,-0.55925,1.66559,-0.58722
4,0.86847,0.97677,1.48163,-0.08568,-0.50966,-0.56254,-0.55925,-0.60039,1.70294
...,...,...,...,...,...,...,...,...,...
931,1.51034,0.97677,-0.29333,-0.90400,-0.50966,-0.56254,-0.55925,-0.60039,1.70294
932,-0.91449,-1.02379,-0.21116,0.73264,-0.50966,1.77764,-0.55925,-0.60039,-0.58722
933,0.86847,-1.02379,0.70919,0.73264,1.96208,1.77764,-0.55925,-0.60039,-0.58722
934,0.08397,-1.02379,-1.39447,0.73264,1.96208,-0.56254,-0.55925,1.66559,-0.58722
